<b> Name : Deepak Gautam <br />
NetID: dg1308 </b>

# RIAK KV

http://basho.com/products/riak-kv/

Riak KV is a distributed NoSQL database with a key/value design and advanced local and multi-cluster replication that guarantees reads and writes even in the event of hardware failures or network partitions. 

Five minuite install: http://docs.basho.com/riak/latest/quickstart/


Download Riak:
http://docs.basho.com/riak/latest/downloads/

For Ubuntu 12:04:
```
sudo apt-get update
sudo apt-get install riak
sudo apt-get install libpam0g-dev
sudo apt-get install libssl0.9.8
wget http://s3.amazonaws.com/downloads.basho.com/riak/2.1/2.1.1/ubuntu/precise/riak_2.1.1-1_amd64.deb
```
Install:

`sudo dpkg -i riak_2.1.1-1_amd64.deb`

Start Riak:

`sudo riak start`

Install Riak Python Client:

`sudo pip install riak`

In [9]:
import riak

### Connecting to Riak

```
myClient = riak.RiakClient(pb_port=8087, protocol='pbc')

# Because the Python client uses the Protocol Buffers interface by
# default, the following will work the same:
myClient = riak.RiakClient(pb_port=8087)
```



In [10]:
myClient = riak.RiakClient(pb_port=8087, protocol='pbc')

### Creating Objects In Riak

First, let’s create a few objects and a bucket to keep them in.

In [11]:
myBucket = myClient.bucket('test')

val1 = 1
key1 = myBucket.new('one', data=val1)
key1.store()

In [12]:
val2 = "two"
key2 = myBucket.new('two', data=val2)
key2.store()

In [13]:
val3 = {"myValue": 3}
key3 = myBucket.new('three', data=val3)
key3.store()

### Reading Objects From Riak

Now that we have a few objects stored, let’s retrieve them and make sure they contain the values we expect.

In [14]:
print myBucket.get('one').data

1


In [15]:
print myBucket.get('one').data
print myBucket.get('two').data
print myBucket.get('three').data

1
two
{'myValue': 3}


### Updating Objects In Riak

While some data may be static, other forms of data may need to be updated. This is also easy to accomplish. Let’s update the value of myValue in the 3rd example to 42.

In [16]:
fetched3 = myBucket.get('three')
fetched3.data["myValue"] = 42
fetched3.store()

### Deleting Objects From Riak

Nothing is complete without a delete. Fortunately, that's easy too.

In [17]:
fetched1 = myBucket.get('one')
fetched2 = myBucket.get('two')
fetched3 = myBucket.get('three')

fetched1.delete()
fetched2.delete()
fetched3.delete()

In [18]:
assert myBucket.get('one').exists == False
assert myBucket.get('two').exists == False
assert myBucket.get('three').exists == False

### Working With Complex Objects

Since the world is a little more complicated than simple integers and bits of strings, let’s see how we can work with more complex objects. Take for example, this object that encapsulates some knowledge about a book.

In [19]:
book = {
  'isbn': "1111979723",
  'title': "Moby Dick",
  'author': "Herman Melville",
  'body': "Call me Ishmael. Some years ago...",
  'copies_owned': 3
}

In [20]:
booksBucket = myClient.bucket('books')
newBook = booksBucket.new(book['isbn'], data=book)
newBook.store()

In [21]:
fetchedBook = booksBucket.get(book['isbn'])

print(fetchedBook.encoded_data)

{"body": "Call me Ishmael. Some years ago...", "author": "Herman Melville", "isbn": "1111979723", "copies_owned": 3, "title": "Moby Dick"}


In [22]:
fetchedBook.delete()

### Same Keys - Different Buckets

The simplest way to split up data would be to use the same identity key across different buckets. A good example of this would be a Customer object, an Order object, and an OrderSummaries object that keeps rolled up info about orders such as Total, etc. Let's put some data into Riak so we can play with it.

In [23]:
customer = {
    'customer_id': 1,
    'name': "John Smith",
    'address': "123 Main Street",
    'city': "Columbus",
    'state': "Ohio",
    'zip': "43210",
    'phone': "+1-614-555-5555",
    'created_date': "2013-10-01 14:30:26"
}

orders = [
    {
        'order_id': 1,
        'customer_id': 1,
        'salesperson_id': 9000,
        'items': [
            {
                'item_id': "TCV37GIT4NJ",
                'title': "USB 3.0 Coffee Warmer",
                'price': 15.99
            },
            {
                'item_id': "PEG10BBF2PP",
                'title': "eTablet Pro, 24GB, Grey",
                'price': 399.99
            }
        ],
        'total': 415.98,
        'order_date': "2013-10-01 14:42:26"
    },
    {
        'order_id': 2,
        'customer_id': 1,
        'salesperson_id': 9001,
        'items': [
            {
                'item_id': "OAX19XWN0QP",
                'title': "GoSlo Digital Camera",
                'price': 359.99
            }
        ],
        'total': 359.99,
        'order_date': "2013-10-15 16:43:16"
    },
    {
        'order_id': 3,
        'customer_id': 1,
        'salesperson_id': 9000,
        'items': [
            {
                'item_id': "WYK12EPU5EZ",
                'title': "Call of Battle: Goats - Gamesphere 4",
                'price': 69.99
            },
            {
                'item_id': "TJB84HAA8OA",
                'title': "Bricko Building Blocks",
                'price': 4.99
            }
        ],
        'total': 74.98,
        'order_date': "2013-11-03 17:45:28"
    }]

order_summary = {
    'customer_id': 1,
    'summaries': [
        {
            'order_id': 1,
            'total': 415.98,
            'order_date': "2013-10-01 14:42:26"
        },
        {
            'order_id': 2,
            'total': 359.99,
            'order_date': "2013-10-15 16:43:16"
        },
        {
            'order_id': 3,
            'total': 74.98,
            'order_date': "2013-11-03 17:45:28"
        }
    ]
}

Create Buckets

In [24]:
client = riak.RiakClient(pb_port=8087, protocol='pbc')
customer_bucket = client.bucket('Customers')
order_bucket = client.bucket('Orders')
order_summary_bucket = client.bucket('OrderSummaries')

Store Data

In [25]:
cr = customer_bucket.new(str(customer['customer_id']),
                         data=customer)
cr.store()

In [26]:
for order in orders:
    order_riak = order_bucket.new(str(order['order_id']),
                                  data=order)
    order_riak.store()

os = order_summary_bucket.new(str(order_summary['customer_id']),
                              data=order_summary)
os.store()

While individual `Customer` and `Order` objects don't change much (or shouldn't change), the Order `Summaries` object will likely change often. It will do double duty by acting as an index for all customer orders, and also holding some relevant data such as the order total, etc. If we showed this information in our application often, it's only one extra request to get all the info. 

In [27]:
customer = customer_bucket.get('1').data
customer['order_summary'] = order_summary_bucket.get('1').data
customer

{'address': '123 Main Street',
 'city': 'Columbus',
 'created_date': '2013-10-01 14:30:26',
 'customer_id': 1,
 'name': 'John Smith',
 'order_summary': {'customer_id': 1,
  'summaries': [{'order_date': '2013-10-01 14:42:26',
    'order_id': 1,
    'total': 415.98},
   {'order_date': '2013-10-15 16:43:16', 'order_id': 2, 'total': 359.99},
   {'order_date': '2013-11-03 17:45:28', 'order_id': 3, 'total': 74.98}]},
 'phone': '+1-614-555-5555',
 'state': 'Ohio',
 'zip': '43210'}

In [28]:
customer_bucket.get('1').delete()